In [ ]:
import numpy
from rbatools.rba_problem_matrix import ProblemMatrix
from rbatools.rba_lp import LinearProblem
from scipy.sparse import coo_matrix

# Solving and manipulating a simple linear optimization problem with rbatools

#### For instructional purposes we use the example-LP provided here:
https://developers.google.com/optimization/lp/lp_example

### Maximize 3x + 4y 
#### s.t.
### x + 2y ≤ 14
### 3x - y ≥ 0
### x - y ≤ 2

    

## Construct solvable LP structure

In [ ]:
print(ProblemMatrix.__doc__)

### Initiate empty Matrix object

In [ ]:
Example_LP_matrix=ProblemMatrix()

### Define lefthand side of LP (constraints) 

In [ ]:
coeffs_constraint_1=[1,2]
coeffs_constraint_2=[3,-1]
coeffs_constraint_3=[1,-1]
Example_LP_matrix.A=coo_matrix(numpy.array([coeffs_constraint_1,coeffs_constraint_2,coeffs_constraint_3]))

### Define righthand side of LP

In [ ]:
Example_LP_matrix.b=numpy.array([14,0,2])

### Define constraint types

In [ ]:
Example_LP_matrix.row_signs=['L','G','L']

### Define objective function 

In [ ]:
Example_LP_matrix.f=numpy.array([3,4])

### Define variable bounds

In [ ]:
Example_LP_matrix.LB=numpy.array([0,0])
Example_LP_matrix.UB=numpy.array([10000,10000])

### Define variable and constraint IDs (columns and rows)

In [ ]:
Example_LP_matrix.col_names=['x','y']
Example_LP_matrix.row_names=['c1','c2','c3']

### Initiate empty LP object and import matrix

In [ ]:
print(LinearProblem.__doc__)

In [ ]:
Example_LP=LinearProblem(lp_solver='swiglpk') # swiglpk or cplex
Example_LP.load_matrix(matrix=Example_LP_matrix)

### Build solvable LP-structure

In [ ]:
lp_built=Example_LP.build_lp() 
if lp_built:
    print("LP built")

## Working with LP

### Optimize LP

In [ ]:
Example_LP.solve_lp()

### Get solution status

In [ ]:
Example_LP.return_solution_status()

### Get optimal objective value

In [ ]:
Example_LP.return_objective_value()

### Get solution vector

In [ ]:
Example_LP.return_primal_values()

### Get shadow prices (dual values)

In [ ]:
Example_LP.return_dual_values()

### There must be a problem, since objective value is 0

The LP is defined as an maximization of the objective function. But by default rbatools assumes minimization.
Therefore the obtained solution corresponds to the minimal feasible objective value.
#### In order to fix the problem we have to invert the objective coefficients (change signs)

### Getting current objective coefficients

In [ ]:
Example_LP.get_objective(['x','y'])

### Imposing objective coefficients with changed signs

In [ ]:
Example_LP.set_objective({'x':-3,'y':-4})

In [ ]:
Example_LP.get_objective(['x','y'])

### Solve LP again

In [ ]:
Example_LP.solve_lp()

### Get new objective value

In [ ]:
Example_LP.return_objective_value()

Objective value is now non-zero.
Please note that it is negative, since we minimized the objective function. 
#### Actual objective value is positive and not negative.

### Corresponding optimal solution:

In [ ]:
Example_LP.return_primal_values()

## What happens if we change the constraint-type of c2 from a >= inequality to an equality? 

In [ ]:
Example_LP.get_constraint_types(['c2'])

In [ ]:
Example_LP.set_constraint_types({'c2': 'E'})

In [ ]:
Example_LP.get_constraint_types(['c2'])

In [ ]:
Example_LP.solve_lp()

In [ ]:
Example_LP.return_solution_status()

In [ ]:
Example_LP.return_objective_value()

In [ ]:
Example_LP.return_primal_values()

## If we also change the constraint-type of c3 from a <= inequality to an equality, the problem becomes infeasible.

In [ ]:
Example_LP.get_constraint_types(['c3'])

In [ ]:
Example_LP.set_constraint_types({'c3': 'E'})

In [ ]:
Example_LP.get_constraint_types(['c3'])

In [ ]:
Example_LP.solve_lp()

In [ ]:
Example_LP.return_solution_status()

### For further operations on the LP (getting/setting variable-bounds, LHS-coefficients, RHS-coefficients ...) please consult the class documentation for:
## rbatools.rba_lp.LinearProblem 